In [1]:
# ETAPA 1: Cargar el PDF, extraer texto y segmentar
# Instala PyMuPDF
!pip install pymupdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 52.8 MB/s eta 0:00:00


In [2]:
import fitz  # PyMuPDF

In [3]:
def extraer_segmentos_desde_pdf(path_pdf, max_tokens=200):
    doc = fitz.open(path_pdf)
    segmentos = []

    for page in doc:
        texto = page.get_text()
        lineas = [linea.strip() for linea in texto.split("\n") if linea.strip()]
        buffer = ""
        for linea in lineas:
            if len((buffer + " " + linea).split()) < max_tokens:
                buffer += " " + linea
            else:
                segmentos.append(buffer.strip())
                buffer = linea
        if buffer:
            segmentos.append(buffer.strip())
    doc.close()
    return segmentos

In [4]:
# Ruta a PDF en Google Drive
ruta_pdf = "/content/drive/MyDrive/Proyectos/ChatBot/Redes_entrada.pdf"
segmentos = extraer_segmentos_desde_pdf(ruta_pdf)
print(f"Segmentos extraídos: {len(segmentos)}")
print("\nEjemplo de segmento:\n", segmentos[0][:500])

Segmentos extraídos: 94

Ejemplo de segmento:
 Resumen Final 6.7 Retrospectiva: un día en la vida de una solicitud de página web Contexto del escenario Un estudiante (Benito) conecta su computadora portátil a la red Ethernet de su facultad y accede a una página web: www.google.com. Aunque parece una operación sencilla, este proceso involucra una gran cantidad de protocolos y capas del modelo TCP/IP. Etapa 1: Obtención de configuración IP vía DHCP Protocolos involucrados: DHCP, UDP, IP, Ethernet​ Capas: Aplicación, Transporte, Red, Enlace 1. 


In [5]:
# ETAPA 2: Vectorización de los segmentos
# Usamos sentence-transformers para convertir los segmentos en embeddings vectoriales.
!pip install sentence-transformers faiss-cpu --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.4 MB/s eta 0:00:00


In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

In [7]:
# Modelo compacto para embeddings
modelo_emb = SentenceTransformer("all-MiniLM-L6-v2")

# Convertir a vectores
embeddings = modelo_emb.encode(segmentos, show_progress_bar=True)

# Crear índice FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

# Guardamos los segmentos asociados
df_segmentos = pd.DataFrame({"texto": segmentos})
print(f" Embeddings generados y guardados.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 Embeddings generados y guardados.


In [8]:
# ETAPA 3: Consulta: Preguntar y recuperar contexto relevante
def recuperar_contexto(pregunta, k=3):
    emb_pregunta = modelo_emb.encode([pregunta])
    distancias, indices = index.search(np.array(emb_pregunta), k)
    resultados = df_segmentos.iloc[indices[0]]
    return "\n".join(resultados["texto"].values)

# Ejemplo
pregunta = "¿Qué es un modelo en desarrollo de software?"
contexto = recuperar_contexto(pregunta)
print(" Contexto recuperado:\n", contexto)

 Contexto recuperado:
 ●​ Un controlador SDN (también llamado sistema operativo de red) ●​ Un conjunto de aplicaciones de control de red El controlador SDN: ●​ Mantiene información del estado de la red (topología, estado de enlaces, hosts, switches, etc.) ●​ Se comunica con las aplicaciones para que estas puedan: ○​ Monitorizar el tráfico ○​ Programar reglas ○​ Controlar los dispositivos de red Una red programable: La programabilidad es el corazón de SDN: ●​ Las aplicaciones de control son el “cerebro” del plano de control. ●​ Estas aplicaciones usan APIs expuestas por el controlador SDN para modificar el comportamiento de la red en tiempo real. ●​ Esto permite aplicar políticas, gestionar tráfico o responder a eventos (fallas, congestión, etc.) de forma centralizada. SDN permite controlar de forma centralizada y programable una red de conmutadores simples, separando el plano de control (inteligencia, decisiones) del plano de datos (reenvío). Esto se logra mediante un controlador SDN y

In [9]:
# ETAPA 4: Generar respuesta con modelo instructivo
from transformers import AutoTokenizer, AutoModelForCausalLM

modelo_llm = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(modelo_llm)
modelo = AutoModelForCausalLM.from_pretrained(modelo_llm)

def responder_con_contexto(pregunta, contexto):
    prompt = f"""[INST] Eres un asistente experto. Responde basándote solo en el siguiente contexto:

{contexto}

Pregunta: {pregunta}
Si no hay información suficiente, responde: "No tengo información suficiente para responder a eso". [/INST]
"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(modelo.device)

    output = modelo.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True).replace(prompt, "").strip()

# Prueba
respuesta = responder_con_contexto(pregunta, contexto)
print(" Respuesta:\n", respuesta)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

 Respuesta:
 [INST] Eres un asistente experto. Responde basándote solo en el siguiente contexto:

●​ Un controlador SDN (también llamado sistema operativo de red) ●​ Un conjunto de aplicaciones de control de red El controlador SDN: ●​ Mantiene información del estado de la red (topología, estado de enlaces, hosts, switches, etc.) ●​ Se comunica con las aplicaciones para que estas puedan: ○​ Monitorizar el tráfico ○​ Programar reglas ○​ Controlar los dispositivos de red Una red programable: La programabilidad es el corazón de SDN: ●​ Las aplicaciones de control son el “cerebro” del plano de control. ●​ Estas aplicaciones usan APIs expuestas por el controlador SDN para modificar el comportamiento de la red en tiempo real. ●​ Esto permite aplicar políticas, gestionar tráfico o responder a eventos (fallas, congestión, etc.) de forma centralizada. SDN permite controlar de forma centralizada y programable una red de conmutadores simples, separando el plano de control (inteligencia, decisiones

In [10]:
# ETAPA 5: Interfaz en Gradio para consultas con recuperación desde PDF
!pip install gradio --quiet

In [11]:
import gradio as gr

def responder_gradio(pregunta, k=3):
    if not pregunta.strip():
        return "Por favor, escribí una pregunta."

    contexto = recuperar_contexto(pregunta, k)
    respuesta = responder_con_contexto(pregunta, contexto)
    return respuesta

In [12]:
gr.Interface(
    fn=responder_gradio,
    inputs=[
        gr.Textbox(label="Tu pregunta"),
        gr.Slider(1, 5, step=1, value=3, label="Cantidad de fragmentos a recuperar")
    ],
    outputs="text",
    title="Asistente basado en PDF",
    description="Hace una pregunta sobre el contenido del PDF. El asistente responderá solo si encuentra información suficiente."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd774f3788703eb843.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
